# Training Data Preparation

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [2]:
import tensorflow as tf
# gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.1,allow_growth=True)
# session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    #Restrict Tensorflow to only allocate 6gb of memory on the first GPU
   try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],
       [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
   except RuntimeError as e:
       #virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# Constants
IMG_SIZE = (299, 299)  # Target image size for InceptionV3

In [4]:
def load_images_from_directory(directory, label):
    """
    Load and preprocess images from the specified directory.
    
    Parameters:
    - directory (str): Path to the directory containing images.
    - label (int): Label for the samples (0 for real, 1 for fake).
    
    Returns:
    - data (np.array): Array of processed images.
    - labels (np.array): Array of corresponding labels.
    """
    data = []
    labels = []

    # Iterate over all images in the directory
    for filename in os.listdir(directory):
        if filename.endswith(('.jpeg','.jpg','.png')):
            filepath = os.path.join(directory, filename)
            img = cv2.imread(filepath)

            if img is not None:
                # Resize and preprocess the image
                img = cv2.resize(img, IMG_SIZE)
                img = img_to_array(img)
                img = preprocess_input(img)  # Preprocess using InceptionV3 preprocessing
                
                data.append(img)
                labels.append(label)
    
    return np.array(data), np.array(labels)

In [5]:
# Load real and fake images
REAL_DIR = "./DFFD/real"
FAKE_DIR = "./DFFD/fake"

x_real, y_real = load_images_from_directory(REAL_DIR, label=0)  # Label 0 for real
x_fake, y_fake = load_images_from_directory(FAKE_DIR, label=1)  # Label 1 for fake

In [6]:
# Combine real and fake data
x_data = np.concatenate([x_real, x_fake], axis=0)
y_data = np.concatenate([y_real, y_fake], axis=0)

In [7]:
# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=42, stratify=y_data)

In [8]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input

In [9]:
# Define the feature extractor using InceptionV3
def create_feature_extractor():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=x)
    return feature_extractor

# Build the model
def create_model():
    feature_extractor = create_feature_extractor()
    
    model = Sequential()
    model.add(Input(shape=(299, 299, 3)))  # Input shape for single images
    model.add(feature_extractor)  # Feature extraction
    model.add(Dense(64, activation='relu'))  # Dense layer for intermediate features
    model.add(Dropout(0.5))  # Dropout for regularization
    model.add(Dense(1, activation='sigmoid'))  # Output layer: binary classification
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
# Create and summarize the model
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2048)              21802784  
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 21,933,985
Trainable params: 21,899,553
Non-trainable params: 34,432
_________________________________________________________________


In [11]:
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=2, validation_data=(x_val, y_val))

Epoch 1/10
854/854 [==============================] - 99s 100ms/step - loss: 0.6826 - accuracy: 0.6516 - val_loss: 545.6284 - val_accuracy: 0.6659
Epoch 2/10
854/854 [==============================] - 86s 101ms/step - loss: 0.6556 - accuracy: 0.6645 - val_loss: 0.6532 - val_accuracy: 0.6659
Epoch 3/10
854/854 [==============================] - 87s 102ms/step - loss: 0.6488 - accuracy: 0.6645 - val_loss: 0.6412 - val_accuracy: 0.6659
Epoch 4/10
854/854 [==============================] - 87s 102ms/step - loss: 0.6447 - accuracy: 0.6645 - val_loss: 0.7053 - val_accuracy: 0.6659
Epoch 5/10
854/854 [==============================] - 87s 102ms/step - loss: 0.6413 - accuracy: 0.6645 - val_loss: 0.6381 - val_accuracy: 0.6659
Epoch 6/10
854/854 [==============================] - 87s 102ms/step - loss: 0.6354 - accuracy: 0.6645 - val_loss: 0.9906 - val_accuracy: 0.6659
Epoch 7/10
854/854 [==============================] - 88s 103ms/step - loss: 0.6459 - accuracy: 0.6645 - val_loss: 0.6370 - val_

In [13]:
# Save the final model
model.save('real1_66%.h5')

In [26]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
import cv2

# Load the saved model
model = load_model('real1_66%.h5')

def preprocess_image(image_path, target_size=(299, 299)):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to load.")
    img = cv2.resize(img, target_size)
    img = img_to_array(img)
    img = preprocess_input(img)  # Preprocess using InceptionV3 preprocessing
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_image_class(image_path, model):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)
    
    # Make prediction
    prediction = model.predict(preprocessed_image)
    
    # Output class
    class_label = 'real' if prediction[0][0] < 0.5 else 'fake'
    print(prediction[0][0])
    return class_label

# Example usage
image_path = 'DFFD/real/Celeb-DF-Real-150.mp4_029.jpeg'
predicted_class = predict_image_class(image_path, model)
print(f'Predicted class for the image: {predicted_class}')


1/1 [==============================] - 1s 750ms/step
0.6676452
Predicted class for the image: fake


In [28]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
import cv2

# Load the saved model
model = load_model('real1_66%.h5')

def preprocess_image(image_path, target_size=(299, 299)):
    """
    Preprocess a single image for model prediction.
    
    Parameters:
    - image_path (str): Path to the image file.
    - target_size (tuple): Desired image size for the model.
    
    Returns:
    - np.array: Preprocessed image ready for prediction.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found or unable to load: {image_path}")
    img = cv2.resize(img, target_size)
    img = img_to_array(img)
    img = preprocess_input(img)  # Preprocess using InceptionV3 preprocessing
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def predict_image_class(image_path, model):
    """
    Predict the class of an image using the trained model.
    
    Parameters:
    - image_path (str): Path to the image file.
    - model (tf.keras.Model): The trained model.
    
    Returns:
    - str: Predicted class label.
    """
    preprocessed_image = preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    class_label = 'real' if prediction[0][0] < 0.5 else 'fake'
    return class_label

def count_images_in_folder(folder_path, model):
    """
    Count the number of images classified as 'real' in the specified folder.
    
    Parameters:
    - folder_path (str): Path to the folder containing images.
    - model (tf.keras.Model): The trained model.
    
    Returns:
    - int: Number of images classified as 'real'.
    """
    real_count = 0
    
    for filename in os.listdir(folder_path):
        if filename.endswith(('.jpeg', '.jpg', '.png')):
            image_path = os.path.join(folder_path, filename)
            try:
                class_label = predict_image_class(image_path, model)
                if class_label == 'real':
                    real_count += 1
            except ValueError as e:
                print(e)  # Log any errors in loading images
    
    return real_count

# Example usage
folder_path = './DFFD/real'
num_real_images = count_images_in_folder(folder_path, model)
print(f'Number of images classified as "real": {num_real_images}')


1/1 [==============================] - 0s 22ms/step
Number of images classified as "real": 0
